In [1]:
import os
import zipfile
import pandas as pd
import joblib
from dateutil import parser
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 1. Data Preparation

In [2]:
# time categories
def get_time_of_day(dt):
    hour = dt.hour
    if hour >=5 and hour < 8:
        return 'Early Morning'
    elif hour >= 8 and hour < 11:
        return 'Morning'
    elif hour >= 11 and hour < 14:
        return 'Midday'
    elif hour >= 14 and hour < 17:
        return 'Afternoon'
    elif hour >= 17 and hour < 20:
        return 'Evening'
    elif hour >= 20 and hour < 23:
        return 'Night'
    else:
        return 'Late Night'
 
# Remove UTC offset from datetime strings
def remove_utc_offset(datetime_str):
    dt = parser.parse(datetime_str)
    return dt.strftime('%Y-%m-%dT%H:%M:%S.%f')

# function to get features
def getfeatures(df):

    df['departuretime'] = df['segmentsDepartureTimeRaw'].apply(remove_utc_offset) 
    df['departuretime'] = pd.to_datetime(df['departuretime'], utc=False)
    
    # time category
    df['time_category'] = df['departuretime'].apply(get_time_of_day)
    
    # departure date
    df['date'] = (df['departuretime'] - pd.Timedelta(hours=2)).dt.date

    # no. of days from flight
    df['days_from_flight'] = (df['date'] - pd.to_datetime(df['searchDate']).dt.date)

    return df

def datefeatures(df):
    # Extract the year
    df['year'] = pd.to_datetime(df['date']).dt.year

    # Extract the month
    df['month'] = pd.to_datetime(df['date']).dt.month

    # Extract the day of the week (Monday=0, Sunday=6)
    df['day_of_week'] = pd.to_datetime(df['date']).dt.dayofweek

    # Extract the day of the month
    df['day_of_month'] = pd.to_datetime(df['date']).dt.day

    return df

def extract_days(duration_string):
    days = re.search(r'(\d+) days', duration_string)
    return int(days.group(1)) if days else None

In [3]:
# Root directory containing subfolders 
root_dir = '../raw/itineraries_csv'

dataframes = []

# Loop through each subfolder
for directory in os.listdir(root_dir):

  if os.path.isdir(os.path.join(root_dir, directory)) and not directory.startswith('.'):
    print(directory)
    folder_path = os.path.join(root_dir, directory)  

    # Loop through zip files
    for filename in os.listdir(folder_path):
      
      if filename.endswith('.zip'):
      
        # Full path of zip file
        zip_path = os.path.join(folder_path, filename)  
        
        # Extract zip contents
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
          zip_ref.extractall(folder_path)
              
          # Parse CSV file in zip 
          csv_path = os.path.join(folder_path, [x for x in zip_ref.namelist() if x.endswith('.csv')][0])      
          df = pd.read_csv(csv_path)
          
          # Apply processing 
          df = df[df['isNonStop'] == True]  
          df = getfeatures(df)
          df = datefeatures(df)

          # median fare
          df = df.groupby(['segmentsDepartureAirportCode', 'segmentsArrivalAirportCode', 'segmentsCabinCode', 'date', 'time_category', 'days_from_flight', 'year', 'month', 'day_of_week', 'day_of_month'])['totalFare'].mean().reset_index(name='meanfare')
          
          # Append to list
          dataframes.append(df) 
        
# Combine dataframes
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.head()

ATL
BOS
CLT
DEN
DFW
DTW
EWR
IAD
JFK
LAX
LGA
MIA
OAK
ORD
PHL
SFO


,segmentsDepartureAirportCode,segmentsArrivalAirportCode,segmentsCabinCode,date,time_category,days_from_flight,year,month,day_of_week,day_of_month,meanfare
0,ATL,BOS,coach,2022-04-17,Afternoon,1 days,2022,4,6,17,398.600000
1,ATL,BOS,coach,2022-04-17,Early Morning,1 days,2022,4,6,17,248.600000
2,ATL,BOS,coach,2022-04-17,Evening,1 days,2022,4,6,17,387.956667
3,ATL,BOS,coach,2022-04-17,Midday,1 days,2022,4,6,17,248.600000
4,ATL,BOS,coach,2022-04-17,Morning,1 days,2022,4,6,17,265.766667


In [4]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1649244 entries, 0 to 1649243
Data columns (total 11 columns):
 #   Column                        Non-Null Count    Dtype          
---  ------                        --------------    -----          
 0   segmentsDepartureAirportCode  1649244 non-null  object         
 1   segmentsArrivalAirportCode    1649244 non-null  object         
 2   segmentsCabinCode             1649244 non-null  object         
 3   date                          1649244 non-null  object         
 4   time_category                 1649244 non-null  object         
 5   days_from_flight              1649244 non-null  timedelta64[ns]
 6   year                          1649244 non-null  int32          
 7   month                         1649244 non-null  int32          
 8   day_of_week                   1649244 non-null  int32          
 9   day_of_month                  1649244 non-null  int32          
 10  meanfare                      1649244 non-null  float6

In [5]:
combined_df['days_from_flight'] = combined_df['days_from_flight'].astype('str')
combined_df['days_from_flight'] = combined_df['days_from_flight'].apply(extract_days)

In [6]:
len(combined_df)

1649244

In [7]:
combined_df.to_csv('meanfares.csv', index=False)

In [8]:
combined_df = pd.read_csv('meanfares.csv')

In [9]:
print('Earliest date:', combined_df['date'].min())
print('Latest date:', combined_df['date'].max())

Earliest date: 2022-04-16
Latest date: 2022-07-18


In [10]:
# Define the split date
split_date = pd.Timestamp('2022-06-17')
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Create the train and test sets
train = combined_df.loc[combined_df.date < split_date]
test = combined_df.loc[combined_df.date >= split_date]

print(len(train))
print(len(test))

1321239
328005


In [11]:
train.head()

,segmentsDepartureAirportCode,segmentsArrivalAirportCode,segmentsCabinCode,date,time_category,days_from_flight,year,month,day_of_week,day_of_month,meanfare
0,ATL,BOS,coach,2022-04-17,Afternoon,1,2022,4,6,17,398.600000
1,ATL,BOS,coach,2022-04-17,Early Morning,1,2022,4,6,17,248.600000
2,ATL,BOS,coach,2022-04-17,Evening,1,2022,4,6,17,387.956667
3,ATL,BOS,coach,2022-04-17,Midday,1,2022,4,6,17,248.600000
4,ATL,BOS,coach,2022-04-17,Morning,1,2022,4,6,17,265.766667


# Modelling

In [17]:
def fitmodel(model, model_name, train=train, test=test):

    X_train = train.drop(columns='meanfare')
    y_train = train['meanfare']

    X_test = test.drop(columns='meanfare')
    y_test = test['meanfare']

    # Define categorical and numeric columns for preprocessing
    categorical_cols = ['segmentsDepartureAirportCode', 'segmentsArrivalAirportCode', 'segmentsCabinCode', 'time_category']
    numeric_cols = ['year',  'month', 'day_of_week', 'day_of_month', 'days_from_flight']

    # Preprocessing and modeling pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numeric_cols), 
            ('cat', OneHotEncoder(), categorical_cols)
        ])

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # Train the model
    pipeline.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = pipeline.predict(X_test)

    # Assuming you have a scikit-learn pipeline named 'pipeline'
    joblib.dump(pipeline, f'../models/{model_name}.pkl')

    # Evaluate the model
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    return pipeline, model_name, rmse, r2, mae, mape

In [18]:
cat = fitmodel(CatBoostRegressor(), 'mean')

Learning rate set to 0.127437
0:	learn: 179.9374142	total: 213ms	remaining: 3m 32s
1:	learn: 171.1806724	total: 398ms	remaining: 3m 18s
2:	learn: 164.1640096	total: 577ms	remaining: 3m 11s
3:	learn: 158.5066143	total: 746ms	remaining: 3m 5s
4:	learn: 154.0248288	total: 888ms	remaining: 2m 56s
5:	learn: 150.3067195	total: 1.07s	remaining: 2m 57s
6:	learn: 147.4399723	total: 1.25s	remaining: 2m 57s
7:	learn: 145.2182631	total: 1.38s	remaining: 2m 51s
8:	learn: 143.4550489	total: 1.55s	remaining: 2m 50s
9:	learn: 141.8569712	total: 1.7s	remaining: 2m 48s
10:	learn: 140.3519657	total: 1.86s	remaining: 2m 47s
11:	learn: 138.8150297	total: 2.03s	remaining: 2m 46s
12:	learn: 137.4489945	total: 2.19s	remaining: 2m 46s
13:	learn: 136.4167756	total: 2.37s	remaining: 2m 47s
14:	learn: 135.5521509	total: 2.54s	remaining: 2m 46s
15:	learn: 134.5228290	total: 2.68s	remaining: 2m 44s
16:	learn: 133.5485251	total: 2.83s	remaining: 2m 43s
17:	learn: 132.7345638	total: 2.98s	remaining: 2m 42s
18:	learn:

In [19]:
print(cat)

(Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['year', 'month',
                                                   'day_of_week',
                                                   'day_of_month',
                                                   'days_from_flight']),
                                                 ('cat', OneHotEncoder(),
                                                  ['segmentsDepartureAirportCode',
                                                   'segmentsArrivalAirportCode',
                                                   'segmentsCabinCode',
                                                   'time_category'])])),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x000001C074572410>)]), 'mean', 91.90098171481256, 0.68097634043311, 63.622884618806225, 0.20962258782321636)


In [15]:
xgboost = fitmodel(XGBRegressor(), 'mean2')

print(xgboost)

C:\Users\dlatl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


(Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['year', 'month',
                                                   'day_of_week',
                                                   'day_of_month',
                                                   'days_from_flight']),
                                                 ('cat', OneHotEncoder(),
                                                  ['segmentsDepartureAirportCode',
                                                   'segmentsArrivalAirportCode',
                                                   'segmentsCabinCode',
                                                   'time_category'])])),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              cols...
                              feature_types=None, gamma=None, grow_policy=None,
            

In [ ]:
import json

unique_segments = combined_df[['segmentsDepartureAirportCode', 'segmentsArrivalAirportCode']].drop_duplicates()

# Create a dictionary where each departure airport has a list of arrival airports
airport_dict = {}
for index, row in unique_segments.iterrows():
    dep_airport = row['segmentsDepartureAirportCode']
    arr_airport = row['segmentsArrivalAirportCode']
    if dep_airport in airport_dict:
        airport_dict[dep_airport].append(arr_airport)
    else:
        airport_dict[dep_airport] = [arr_airport]

# Convert dictionary to JSON
json_output = json.dumps(airport_dict, indent=4)

# Printing or using the json_output as needed
print(json_output)


{
    "OAK": [
        "DEN",
        "LAX",
        "PHL",
        "DTW",
        "ORD"
    ],
    "IAD": [
        "SFO",
        "ATL",
        "BOS",
        "CLT",
        "DEN",
        "DFW",
        "DTW",
        "EWR",
        "JFK",
        "LAX",
        "LGA",
        "MIA",
        "ORD"
    ],
    "DEN": [
        "ATL",
        "BOS",
        "CLT",
        "DFW",
        "DTW",
        "EWR",
        "IAD",
        "JFK",
        "LAX",
        "LGA",
        "MIA",
        "OAK",
        "ONT",
        "ORD",
        "PHL",
        "SFO"
    ],
    "LGA": [
        "ATL",
        "BOS",
        "CLT",
        "DEN",
        "DFW",
        "DTW",
        "IAD",
        "MIA",
        "ORD",
        "LAX"
    ],
    "LAX": [
        "ATL",
        "BOS",
        "CLT",
        "DAL",
        "DEN",
        "DFW",
        "DTW",
        "EWR",
        "IAD",
        "JFK",
        "LGA",
        "MIA",
        "OAK",
        "ORD",
        "PHL",
        "SFO"
    ],
   

In [ ]:
with open('flightroutes.json', 'w') as file:
    file.write(json_output)